# What needs doing:

* Count all words and sentences from all terms ✓
* Get term limits from wikipedia ✓
* Count occurences of body, desc, kinesic... items ✓

# Questions for Tomaž/Nikola:

* How is validation run?
* Do we need to follow the per-term directory and file structure (e.g. ParlaSpeech-HR) or can we preserve the current structure?
* What is the deal with relations, coalition field `mutual` for party listing while opposition has `active` and a field `passive #government.hr`?
* I don't have the education title (e.g. `<education n="6">stručni prvostupnik sestrinstva</education>`), only years of education.
* I don't have the exact dates of terms. I can only infer them from the rest of the data I have (I estimate it from the dates of speeches.)
* I get different data for coalition-opposition formations than in the existing main TEI files for term 9.
* I need the affiliation>roles, but they are not in the MP datasheet.

Tomaž:
* Education titles and `n` parameter?
* How to date ongoing terms?

In [58]:
from typing import Union, List
from pathlib import Path
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import pandas as pd

mpdf = pd.read_pickle("mpdf_corrected")
partiesdf = pd.read_pickle("partiesdf_corrected")
termdata = pd.read_csv("../terms.csv").set_index("Term")

def transform_abbreviation(s: str) -> str:
    return f"party.{s.replace(' ', '_').upper()}"


In [59]:
# set(mpdf.party.unique()) - set(partiesdf.party.unique())
mpdf.party[~mpdf.party.isin(partiesdf.party)].unique()

array(['nezavisni', 'HKS', 'HS'], dtype=object)

# Preparing persons:

In [60]:
from xml.etree.ElementTree import Element, SubElement, tostring, XML, parse
from xml.dom import minidom
def pretty_print(s) -> None:
    print(minidom.parseString(tostring(s).decode("utf")).toprettyxml("\t"))
def pretty_string(s) -> None:
    return minidom.parseString(tostring(s).decode("utf")).toprettyxml("\t")

In [61]:
partiesdf.party.unique()

array(['DC', 'HDSSB', 'HDZ', 'HNS', 'HSLS', 'HSP', 'HSS', 'HSU', 'IDS',
       'LIBRA', 'LS', 'MDS', 'PGS', 'SBHS', 'SDAH', 'SDP', 'SDSS',
       'Hrvatski laburisti', 'HSD', 'BDSH', 'HGS', 'HSP AS', 'ID',
       'Naprijed Hrvatska!', 'Novi val', 'ORaH', 'Reformisti', 'BM365',
       'BUZ', 'HDS', 'HRAST', 'HRID', 'MOST', 'NLSP', 'PH', 'Živi zid',
       'GLAS', 'SMSH', 'SNAGA', 'DP', 'ZK', 'SIP', 'Pametno', 'Fokus'],
      dtype=object)

In [62]:
termdata

,From,To
Term,,
5,2003-12-22,2007-10-12
6,2008-01-11,2011-10-28
7,2011-12-22,2015-09-25
8,2015-12-03,2016-07-15
9,2016-10-14,2022-05-18
10,2022-07-22,ongoing


# Terms (line 438):

path: `/teiCorpus/teiHeader/profileDesc/particDesc/listOrg/org[2]/listEvent`

In [63]:
listEvent = Element("listEvent")
head = SubElement(listEvent, "head")
head.set("xml:lang", "hr")
head.text = "Mandatno obdoblje"
head = SubElement(listEvent, "head")
head.set("xml:lang", "en")
head.text = "Legislative period"

for i, row in termdata.iterrows():
    event = SubElement(listEvent, "event")
    event.set("xml:id", f"HS.{i}")
    event.set("from", row["From"])
    event.set("to", row["To"])
    
    label = SubElement(event, "label")
    label.set("xml:lang", "hr")
    label.text = f"{i}. mandat"
    label = SubElement(event, "label")
    label.set("xml:lang", "en")
    label.text = f"Term {i}"
listEvent_str = pretty_string(listEvent)
listEvent_str = "\n".join(listEvent_str.split("\n")[1:])
# print(listEvent_str)

# Parties: line 447

path:
`/teiCorpus/teiHeader/profileDesc/particDesc/listOrg/org[3:]`

In [64]:
"Hrvatski laburisti" in mpdf.party

False

In [65]:
partiesdf["id"] = partiesdf.party.apply(transform_abbreviation)

partydata = partiesdf["party full_name id".split()].drop_duplicates()

orgs = []
for i, row in partydata.iterrows():
    org = Element("org")
    org.set("xml:id", row["id"])
    org.set("role", "parliamentaryGroup")
    orgName = SubElement(org, "orgName")
    orgName.set("full", "yes")
    orgName.set("xml:lang", "hr")
    orgName.text = row["full_name"]
    
    orgName = SubElement(org, "orgName")
    orgName.set("full", "abb")
    orgName.text = row["party"]
    orgs.append(org)
    
# Add nezavisni:
org = Element("org")
org.set("xml:id", "party.NEZAVISNI")
org.set("role", "parliamentaryGroup")
orgName = SubElement(org, "orgName")
orgName.set("full", "yes")
orgName.set("xml:lang", "hr")
orgName.text = "nezavisni"
orgName = SubElement(org, "orgName")
orgName.set("full", "abb")
orgName.text = "nezavisni"
orgs.append(org)

# Add Hrvatski suverenisti:
org = Element("org")
org.set("xml:id", "party.HS")
org.set("role", "parliamentaryGroup")
orgName = SubElement(org, "orgName")
orgName.set("full", "yes")
orgName.set("xml:lang", "hr")
orgName.text = "Hrvatski Suverenisti"
orgName = SubElement(org, "orgName")
orgName.set("full", "abb")
orgName.text = "HS"
orgs.append(org)

# Add Hrvatska konzervativna stranka
org = Element("org")
org.set("xml:id", "party.HKS")
org.set("role", "parliamentaryGroup")
orgName = SubElement(org, "orgName")
orgName.set("full", "yes")
orgName.set("xml:lang", "hr")
orgName.text = "Hrvatska konzervativna stranka"
orgName = SubElement(org, "orgName")
orgName.set("full", "abb")
orgName.text = "HKS"
orgs.append(org)


orgs = [pretty_string(i) for i in orgs]

orgstring = "\n".join(
    ["\n".join(org.split("\n")[1:])for org in orgs]
)
# print(orgstring)

# People: line 525

path: `/teiCorpus/teiHeader/profileDesc/particDesc/listPerson`

In [66]:
for name in mpdf.fullname.unique():
    if mpdf.loc[mpdf.fullname==name, "codemp"].unique().shape[0] != 1:
        print(name, "\n", mpdf.loc[mpdf.fullname==name, "codemp"])

Vrbat Grgić, Tanja 
 334    M581
524    M582
725    M582
Name: codemp, dtype: object
Lovrić Merzel, Marina 
 376    M393
461    M394
Name: codemp, dtype: object


In [67]:
listPerson = []
codemps = mpdf.codemp.drop_duplicates()
n = 0
for i, c in enumerate(codemps):    
    subset = mpdf.query(f"codemp=='{c}'").reset_index(drop=True)
    surname = subset.lastname[0]
    forename = subset.firstname[0]
    sex = "M" if subset.gender[0] == 0 else "F"
    birth = str(subset.date_of_birth[0])
    birth = f"{birth[0:4]}-{birth[4:6]}-{birth[-2:]}"
    reference_name = f"{surname.replace(' ', '')}{forename.replace(' ', '')}"
    
    person = Element("person")
    person.set("xml:id", reference_name)
    person.set("n", str(n))
    n += 1
    persName = SubElement(person, "persName")
    
    buf = SubElement(persName, "surname")
    buf.text = surname
    
    buf = SubElement(persName, "forename")
    buf.text = forename
    
    buf = SubElement(person, "sex")
    buf.set("value", sex)
    
    buf = SubElement(person, "birth")
    buf.set("when", birth)
    

    for i, row in subset.drop_duplicates().iterrows():        
        ana = SubElement(person, "affiliation")
        ana.set("role", "member")
        ana.set("ref", f"#{transform_abbreviation(row['party'])}")
        
        rolename = SubElement(ana, "roleName")
        rolename.set("xml:lang", "en")
        rolename.text = "Member"

        ana = SubElement(person, "affiliation")
        ana.set("role", "member")
        ana.set("ref", "#HS")
        ana.set("ana", f"#HS.{row['term2']}")
        rolename = SubElement(ana, "roleName")
        rolename.set("xml:lang", "en")
        rolename.text = "MP"
    listPerson.append(person)


In [ ]:
listPerson = ["\n".join(pretty_string(p).split("\n")[1:]) for p in listPerson]
listPerson = "\n".join(listPerson)


In [ ]:
path_to_term_tmxs = Path("../T/")
assert path_to_term_tmxs.exists()


xiincludes = ""
for file in sorted(list(path_to_term_tmxs.glob("ParlaMint-HR_T*.xml"))):
    xiincludes += f"""    <xi:include xmlns:xi="http://www.w3.org/2001/XInclude" href="{file.name}"/>\n"""


# Relations

path: `/teiCorpus/teiHeader/profileDesc/particDesc/listOrg/listRelation`

In [ ]:
gb = partiesdf.groupby(["term2", "coalition"]).agg(
    {
        "id": lambda l: " ".join(set(["#"+i for i in l]))
    }
)
terms = partiesdf.term2.unique()
listRelation = ""
for term in terms:
    fromto = termdata.loc[term, :]
    ind = (term, 1)
    coalition = gb.loc[ind].id
    ind = (term, 0)
    opposition = gb.loc[ind].id
    listRelation += f"""
    <relation name="coalition"
            mutual="{coalition}"
            from="{fromto.From}"
            to="{fromto.To}"
            ana="#HS.{term}"/>
    <relation name="oposition"
            active="{opposition}"
            passive="#government.HR"
            from="{fromto.From}"
            to="{fromto.To}"
            ana="#HS.{term}"/>\n"""    

# Counting extents and tagUsages

In [ ]:
from typing import Union
from pathlib import Path
def get_extent(path: Union[str, Path]):
    if isinstance(path, Path):
        assert path.exists()
        path = str(path)
    namespace= {
        "n": "http://www.tei-c.org/ns/1.0",
        
    }
    termtei = parse(path)
    measures = termtei.getroot().findall(".//n:measure", namespaces=namespace)
    for i in measures:
        if i.get("unit") == "speeches":
            speeches = i.get("quantity")
        if i.get("unit") == "words":
            words = i.get("quantity")
    return {"words": int(words), "speeches": int(speeches)}

def get_tagusage(path: Union[str, Path]):
    if isinstance(path, Path):
        assert path.exists()
        path = str(path)
    namespace= {
        "n": "http://www.tei-c.org/ns/1.0",
        
    }
    termtei = parse(path)
    tagusages = termtei.getroot().findall(".//n:tagUsage", namespaces=namespace)
    for i in tagusages:
        if i.get("unit") == "speeches":
            speeches = i.get("quantity")
        if i.get("unit") == "words":
            words = i.get("quantity")
    return {i.get("gi"): i.get("occurs") for i in tagusages}

path_to_term_tmxs = Path("../T/")
assert path_to_term_tmxs.exists()


extents = []
tagusages = []
for file in sorted(list(path_to_term_tmxs.glob("ParlaMint-HR_T*.xml"))):
    extents.append(get_extent(file))
    tagusages.append(get_tagusage(file))

tagusage = ""
import pandas as pd
tus = pd.DataFrame(data=tagusages).astype(int)
for gi, occurs in tus.sum().items():
    tagusage += f"""<tagUsage gi="{gi}" occurs="{occurs}"/>\n"""
extent_count = pd.DataFrame(data=extents).astype(int).sum()
words = extent_count["words"]
speeches = extent_count["speeches"]
extent = f"""<measure unit="speeches" quantity="{speeches}" xml:lang="hr">{f'{speeches:,d}'.replace(',','.')} govora</measure>
<measure unit="speeches" quantity="{speeches}" xml:lang="en">{speeches:,d} speeches</measure>
<measure unit="words" quantity="{words}" xml:lang="hr">{f'{words:,d}'.replace(',','.')} riječi</measure>
<measure unit="words" quantity="{words}" xml:lang="en">{words:,d} words</measure>
"""

# Putting it all together

In [ ]:
with open("../T/ParlaMint-HR_template2.xml") as f:
    content = f.read()
from string import Template
template = Template(content)
filled_in = template.substitute(
                    listEvent=listEvent_str,
                    orgs=orgstring,
                    listPerson = listPerson,
                    xiincludes = xiincludes,
                    listRelation = listRelation,
                    extent = extent,
                    tagusage = tagusage
                    )

with open("../T/ParlaMint-HR.xml", "w") as f:
    f.write(
        filled_in
    )